In [165]:
##nodejs:  https://docs.aws.amazon.com/sdk-for-javascript/v2/developer-guide/setting-up-node-on-ec2-instance.html

# !pip install "jupyterlab>=3" "ipywidgets>=7.6"
# !pip install jupyter-dash
# !jupyter lab build


# !pip install snowflake --user
# !pip install snowflake-connector-python --user
# !pip install category_encoders
# !pip install xgboost
# !pip install lightgbm --user
import os
import sys
path=!pwd
sys.path.append(os.path.join(path[0], '..'))
from utils import *
import snowflake.connector
from datetime import timedelta

from category_encoders import OneHotEncoder
import xgboost as xgb
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as r2_score
import sklearn.model_selection

class SnowflakeConnector(BaseConnector):
    def __init__(self, credentials: Credentials):
        keys = credentials.get_keys()
        self._secrets = json.loads(keys.get('SecretString', "{}"))

    def connect(self, dbname: str, schema: str = 'DEFAULT'):
        ctx = snowflake.connector.connect(
            user=self._secrets['login_name'],
            password=self._secrets['login_password'],
            account=self._secrets['account'],
            warehouse=self._secrets['warehouse'],
            database=dbname,
            schema=schema
        )

        return ctx
    
## Credentials
SF_CREDS = 'datascience-max-dev-sagemaker-notebooks'

## Snowflake connection 
conn=SnowflakeConnector(SSMPSCredentials(SF_CREDS))
ctx=conn.connect("MAX_PROD","DATASCIENCE_STAGE")
cur = ctx.cursor()

## Assets_dim

In [49]:
querystr = '''
select * from max_prod.catalog.asset_dim a
join max_prod.catalog.reporting_asset_offering_dim raod
on a.viewable_id = raod.viewable_id
and brand = 'HBO MAX'
and territory = 'HBO MAX DOMESTIC'
and channel = 'HBO MAX SUBSCRIPTION'
right outer join
(select title, tier, season, category, premiere_date from max_prod.content_analytics.daily_future_programming_schedule group by title, tier, season, category, premiere_date) fp
on a.asset_title_short = fp.title
and zeroifnull(a.season_number) = zeroifnull(fp.season)
;
'''

cursor_list = ctx.execute_string(
    querystr
    )
df = pd.DataFrame.from_records(cursor_list[-1].fetchall(), columns=[x[0] for x in cursor_list[-1].description])
df.columns= df.columns.str.lower()
display(df.head(2))
display(df.shape)
df.to_csv('s3://datascience-hbo-users/users/tjung/psi/future_program_metadata.csv')

,asset_id,wm_id,viewable_id,source_asset_id,series_id,season_id,franchise_id,catalog_geo_id,imdb_id,csm_id,grace_notes_id,asset_title_long,asset_title_short,asset_summary_long,asset_summary_short,asset_tags_age,asset_tags_content_category,asset_tags_genre,asset_tags_other,brand_hubs,asset_type,catalog_asset_type,series_title_long,series_title_short,series_type,season_title_long,season_title_short,promoted_asset_id,season_number,season_premiere_ind,season_finale_ind,series_premiere_ind,episode_number_in_season,episode_number_in_series,program_type,content_category,primary_genre_desc,navigation_genre_desc,descriptive_genre_desc,wm_enterprise_genres,leaf_level_genres,animation_type,script_type,scripted_flag,sports_flag,kids_flag,international_flag,latino_flag,short_flag,age_group,original_network,release_year,license_contract_type,license_contract_term,licensor,original_language,first_offered_date_hbo,first_offered_date_now,first_offered_date_max,air_date,latest_availability_start_date,latest_availability_end_date,premiere_air_date,theatrical_release_date,original_linear_air_date,full_asset_tags_age,full_asset_tags_content_category,full_asset_tags_genre,full_asset_tags_other,full_brand_hubs,is_downloadable,asset_url,creation_time,last_modified_time,last_rollup_trigger_time,effective_start_date,effective_end_date,active_flag,title_id,viewable_id,start_utc,end_utc,latest_asset_release_date,season_first_offered_date,title_first_offered_date,first_offered_date,latest_availability_start_date,latest_availability_end_date,brand,territory,channel,ever_popcorn_title,creation_time,last_modified_time,title,tier,season,category,premiere_date
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,None,None,NaT,NaT,NaT,None,NaT,None,NaT,NaT,NaT,None,None,None,None,None,NaN,None,NaT,NaT,NaT,NaT,None,NaN,None,None,NaT,NaT,NaT,NaT,NaT,NaT,NaT,None,None,None,None,NaN,NaT,NaT,Legendary,3,3,Unscripted Series,2022-05-05
1,6880571200656194905,urn:warnermedia:wmid:product:714a9d96bfcb15d26...,GXodE-Q8aYbDCYwEAAAQT,PROD6988,None,None,None,None,tt0087182,4a5cb47d-dc8d-478d-8502-ecd33e9609fc,None,Dune,Dune,The year: 10191. Two factions war bitterly ove...,Two factions in the year 10191 war bitterly ov...,None,movie,fantasy-sci-fi,action | feature | horror-sci-fi | movie | movies,hbo | turner-classic-movies,FEATURE,movie,None,None,None,None,None,None,NaN,0.0,0.0,0.0,NaN,NaN,acquired,movies,action,Fantasy & Sci-Fi,Sci-Fi|Adventure,action | fantasy & sci-fi,action | adventure | fantasy | sci-fi,Live Action,Scripted,1.0,0.0,0.0,0.0,0.0,0.0,None,HBO,1984.0,LIBRARY,3,"UNIVERSAL CITY STUDIOS PRODUCTIONS, LLLP",None,2020-06-01 10:00:00,2020-06-01 10:00:00,2020-06-01 10:00:00,1984-12-14,2020-06-01 12:00:00,2021-11-30 12:00:00,1984-12-14 12:00:00,1984-12-14,1985-11-09 20:00:00,None,urn:hbo:content-category:movie,urn:hbo:genre:fantasy-sci-fi,urn:tag:action | urn:tag:feature | urn:tag:hor...,urn:warnermedia:brand:hbo | urn:warnermedia:br...,0.0,/movies/GXodE-Q8aYbDCYwEAAAQT,2020-10-16 22:52:56.548,2021-10-05 14:37:17.321,2020-10-16 22:52:56.548,2020-10-16 22:52:56.548,2999-12-31 00:00:00,1.0,GXodE-Q8aYbDCYwEAAAQT,GXodE-Q8aYbDCYwEAAAQT,2020-06-01 10:00:00,2021-12-01 10:59:00,2020-06-01 10:00:00,NaT,2020-06-01 10:00:00,2020-06-01 10:00:00,2020-06-01 12:00:00,2021-11-30 12:00:00,HBO MAX,HBO MAX DOMESTIC,HBO MAX SUBSCRIPTION,0.0,2021-10-06 00:42:02.338,2021-10-06 00:42:02.338,Dune,1,0,Popcorn,2021-10-22


(857, 99)

In [4]:
df = pd.read_csv('s3://datascience-hbo-users/users/tjung/psi/future_program_metadata.csv')
names = df.columns.tolist()
df = df[names[-5:] + names[:-5]]
df['premiere_date'] = pd.to_datetime(df['premiere_date'])
df['premiere_year_month'] = df['premiere_date'].dt.strftime('%Y-%m')
display(df.head(2))
print(df.shape)
print(df.title.nunique())
print(df.isnull().sum())


,title,tier,season,category,premiere_date,Unnamed: 0,asset_id,wm_id,viewable_id,source_asset_id,series_id,season_id,franchise_id,catalog_geo_id,imdb_id,csm_id,grace_notes_id,asset_title_long,asset_title_short,asset_summary_long,asset_summary_short,asset_tags_age,asset_tags_content_category,asset_tags_genre,asset_tags_other,brand_hubs,asset_type,catalog_asset_type,series_title_long,series_title_short,series_type,season_title_long,season_title_short,promoted_asset_id,season_number,season_premiere_ind,season_finale_ind,series_premiere_ind,episode_number_in_season,episode_number_in_series,program_type,content_category,primary_genre_desc,navigation_genre_desc,descriptive_genre_desc,wm_enterprise_genres,leaf_level_genres,animation_type,script_type,scripted_flag,sports_flag,kids_flag,international_flag,latino_flag,short_flag,age_group,original_network,release_year,license_contract_type,license_contract_term,licensor,original_language,first_offered_date_hbo,first_offered_date_now,first_offered_date_max,air_date,latest_availability_start_date,latest_availability_end_date,premiere_air_date,theatrical_release_date,original_linear_air_date,full_asset_tags_age,full_asset_tags_content_category,full_asset_tags_genre,full_asset_tags_other,full_brand_hubs,is_downloadable,asset_url,creation_time,last_modified_time,last_rollup_trigger_time,effective_start_date,effective_end_date,active_flag,title_id,viewable_id.1,start_utc,end_utc,latest_asset_release_date,season_first_offered_date,title_first_offered_date,first_offered_date,latest_availability_start_date.1,latest_availability_end_date.1,brand,territory,channel,ever_popcorn_title,creation_time.1,last_modified_time.1,premiere_year_month
0,Legendary,3,3,Unscripted Series,2022-05-05,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05
1,Dune,1,0,Popcorn,2021-10-22,1,6.880571e+18,urn:warnermedia:wmid:product:714a9d96bfcb15d26...,GXodE-Q8aYbDCYwEAAAQT,PROD6988,NaN,NaN,NaN,NaN,tt0087182,4a5cb47d-dc8d-478d-8502-ecd33e9609fc,NaN,Dune,Dune,The year: 10191. Two factions war bitterly ove...,Two factions in the year 10191 war bitterly ov...,NaN,movie,fantasy-sci-fi,action | feature | horror-sci-fi | movie | movies,hbo | turner-classic-movies,FEATURE,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,acquired,movies,action,Fantasy & Sci-Fi,Sci-Fi|Adventure,action | fantasy & sci-fi,action | adventure | fantasy | sci-fi,Live Action,Scripted,1.0,0.0,0.0,0.0,0.0,0.0,NaN,HBO,1984.0,LIBRARY,3.0,"UNIVERSAL CITY STUDIOS PRODUCTIONS, LLLP",NaN,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,1984-12-14,2020-06-01 12:00:00.000,2021-11-30 12:00:00,1984-12-14 12:00:00.000,1984-12-14 00:00:00.000,1985-11-09 20:00:00.000,NaN,urn:hbo:content-category:movie,urn:hbo:genre:fantasy-sci-fi,urn:tag:action | urn:tag:feature | urn:tag:hor...,urn:warnermedia:brand:hbo | urn:warnermedia:br...,0.0,/movies/GXodE-Q8aYbDCYwEAAAQT,2020-10-16 22:52:56.548,2021-10-05 14:37:17.321,2020-10-16 22:52:56.548,2020-10-16 22:52:56.548,2999-12-31 00:00:00,1.0,GXodE-Q8aYbDCYwEAAAQT,GXodE-Q8aYbDCYwEAAAQT,2020-06-01 10:00:00.000,2021-12-01 10:59:00.000,2020-06-01 10:00:00.000,NaN,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,2020-06-01 12:00:00.000,2021-11-30 12:00:00,HBO MAX,HBO MAX DOMESTIC,HBO MAX SUBSCRIPTION,0.0,2021-10-06 00:42:02.338,2021-10-06 00:42:02.338,2021-10


(857, 101)
640
title                     0
tier                      0
season                    0
category                  0
premiere_date             0
                       ... 
channel                 761
ever_popcorn_title      761
creation_time.1         761
last_modified_time.1    761
premiere_year_month       0
Length: 101, dtype: int64


In [71]:
df[(df.category=='Popcorn') & (df.premiere_year_month>='2021-10')].sort_values(by='premiere_date')

,title,tier,season,category,premiere_date,Unnamed: 0,asset_id,wm_id,viewable_id,source_asset_id,series_id,season_id,franchise_id,catalog_geo_id,imdb_id,csm_id,grace_notes_id,asset_title_long,asset_title_short,asset_summary_long,asset_summary_short,asset_tags_age,asset_tags_content_category,asset_tags_genre,asset_tags_other,brand_hubs,asset_type,catalog_asset_type,series_title_long,series_title_short,series_type,season_title_long,season_title_short,promoted_asset_id,season_number,season_premiere_ind,season_finale_ind,series_premiere_ind,episode_number_in_season,episode_number_in_series,program_type,content_category,primary_genre_desc,navigation_genre_desc,descriptive_genre_desc,wm_enterprise_genres,leaf_level_genres,animation_type,script_type,scripted_flag,sports_flag,kids_flag,international_flag,latino_flag,short_flag,age_group,original_network,release_year,license_contract_type,license_contract_term,licensor,original_language,first_offered_date_hbo,first_offered_date_now,first_offered_date_max,air_date,latest_availability_start_date,latest_availability_end_date,premiere_air_date,theatrical_release_date,original_linear_air_date,full_asset_tags_age,full_asset_tags_content_category,full_asset_tags_genre,full_asset_tags_other,full_brand_hubs,is_downloadable,asset_url,creation_time,last_modified_time,last_rollup_trigger_time,effective_start_date,effective_end_date,active_flag,title_id,viewable_id.1,start_utc,end_utc,latest_asset_release_date,season_first_offered_date,title_first_offered_date,first_offered_date,latest_availability_start_date.1,latest_availability_end_date.1,brand,territory,channel,ever_popcorn_title,creation_time.1,last_modified_time.1,premiere_year_month
418,Many Saints of Newark,2,0,Popcorn,2021-10-15,418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10
1,Dune,1,0,Popcorn,2021-10-22,1,6.880571e+18,urn:warnermedia:wmid:product:714a9d96bfcb15d26...,GXodE-Q8aYbDCYwEAAAQT,PROD6988,NaN,NaN,NaN,NaN,tt0087182,4a5cb47d-dc8d-478d-8502-ecd33e9609fc,NaN,Dune,Dune,The year: 10191. Two factions war bitterly ove...,Two factions in the year 10191 war bitterly ov...,NaN,movie,fantasy-sci-fi,action | feature | horror-sci-fi | movie | movies,hbo | turner-classic-movies,FEATURE,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,acquired,movies,action,Fantasy & Sci-Fi,Sci-Fi|Adventure,action | fantasy & sci-fi,action | adventure | fantasy | sci-fi,Live Action,Scripted,1.0,0.0,0.0,0.0,0.0,0.0,NaN,HBO,1984.0,LIBRARY,3.0,"UNIVERSAL CITY STUDIOS PRODUCTIONS, LLLP",NaN,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,1984-12-14,2020-06-01 12:00:00.000,2021-11-30 12:00:00,1984-12-14 12:00:00.000,1984-12-14 00:00:00.000,1985-11-09 20:00:00.000,NaN,urn:hbo:content-category:movie,urn:hbo:genre:fantasy-sci-fi,urn:tag:action | urn:tag:feature | urn:tag:hor...,urn:warnermedia:brand:hbo | urn:warnermedia:br...,0.0,/movies/GXodE-Q8aYbDCYwEAAAQT,2020-10-16 22:52:56.548,2021-10-05 14:37:17.321,2020-10-16 22:52:56.548,2020-10-16 22:52:56.548,2999-12-31 00:00:00,1.0,GXodE-Q8aYbDCYwEAAAQT,GXodE-Q8aYbDCYwEAAAQT,2020-06-01 10:00:00.000,2021-12-01 10:59:00.000,2020-06-01 10:00:00.000,NaN,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,2020-06-01 12:00:00.000,2021-11-30 12:00:00,HBO MAX,HBO MAX DOMESTIC,HBO MAX SUBSCRIPTION,0.0,2021-10-06 00:42:02.338,2021-10-06 00:42:02.338,2021-10
2,Dune,1,0,Popcorn,2021-10-22,2,7.549632e+18,urn:warnermedia:wmid:product:df59373b53ac93d41...,GYUjdLgBiJp5otAEAAAAJ,2239504,NaN,NaN,NaN,NaN,tt1160419,NaN,NaN,Dune,Dune,"Oscar nominee Denis Villeneuve (""Arrival,"" ""Bl...",A mythic and emotionally charged hero's journe...,teen,movie,adventure | drama | fantasy-sci-fi | ki

In [70]:
df[(df.tier==1) & (df.premiere_year_month>='2021-10')].sort_values(by='premiere_date')

,title,tier,season,category,premiere_date,Unnamed: 0,asset_id,wm_id,viewable_id,source_asset_id,series_id,season_id,franchise_id,catalog_geo_id,imdb_id,csm_id,grace_notes_id,asset_title_long,asset_title_short,asset_summary_long,asset_summary_short,asset_tags_age,asset_tags_content_category,asset_tags_genre,asset_tags_other,brand_hubs,asset_type,catalog_asset_type,series_title_long,series_title_short,series_type,season_title_long,season_title_short,promoted_asset_id,season_number,season_premiere_ind,season_finale_ind,series_premiere_ind,episode_number_in_season,episode_number_in_series,program_type,content_category,primary_genre_desc,navigation_genre_desc,descriptive_genre_desc,wm_enterprise_genres,leaf_level_genres,animation_type,script_type,scripted_flag,sports_flag,kids_flag,international_flag,latino_flag,short_flag,age_group,original_network,release_year,license_contract_type,license_contract_term,licensor,original_language,first_offered_date_hbo,first_offered_date_now,first_offered_date_max,air_date,latest_availability_start_date,latest_availability_end_date,premiere_air_date,theatrical_release_date,original_linear_air_date,full_asset_tags_age,full_asset_tags_content_category,full_asset_tags_genre,full_asset_tags_other,full_brand_hubs,is_downloadable,asset_url,creation_time,last_modified_time,last_rollup_trigger_time,effective_start_date,effective_end_date,active_flag,title_id,viewable_id.1,start_utc,end_utc,latest_asset_release_date,season_first_offered_date,title_first_offered_date,first_offered_date,latest_availability_start_date.1,latest_availability_end_date.1,brand,territory,channel,ever_popcorn_title,creation_time.1,last_modified_time.1,premiere_year_month
631,Succession,1,3,Scripted Drama Series,2021-10-17,631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10
1,Dune,1,0,Popcorn,2021-10-22,1,6.880571e+18,urn:warnermedia:wmid:product:714a9d96bfcb15d26...,GXodE-Q8aYbDCYwEAAAQT,PROD6988,NaN,NaN,NaN,NaN,tt0087182,4a5cb47d-dc8d-478d-8502-ecd33e9609fc,NaN,Dune,Dune,The year: 10191. Two factions war bitterly ove...,Two factions in the year 10191 war bitterly ov...,NaN,movie,fantasy-sci-fi,action | feature | horror-sci-fi | movie | movies,hbo | turner-classic-movies,FEATURE,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,acquired,movies,action,Fantasy & Sci-Fi,Sci-Fi|Adventure,action | fantasy & sci-fi,action | adventure | fantasy | sci-fi,Live Action,Scripted,1.0,0.0,0.0,0.0,0.0,0.0,NaN,HBO,1984.0,LIBRARY,3.0,"UNIVERSAL CITY STUDIOS PRODUCTIONS, LLLP",NaN,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,1984-12-14,2020-06-01 12:00:00.000,2021-11-30 12:00:00,1984-12-14 12:00:00.000,1984-12-14 00:00:00.000,1985-11-09 20:00:00.000,NaN,urn:hbo:content-category:movie,urn:hbo:genre:fantasy-sci-fi,urn:tag:action | urn:tag:feature | urn:tag:hor...,urn:warnermedia:brand:hbo | urn:warnermedia:br...,0.0,/movies/GXodE-Q8aYbDCYwEAAAQT,2020-10-16 22:52:56.548,2021-10-05 14:37:17.321,2020-10-16 22:52:56.548,2020-10-16 22:52:56.548,2999-12-31 00:00:00,1.0,GXodE-Q8aYbDCYwEAAAQT,GXodE-Q8aYbDCYwEAAAQT,2020-06-01 10:00:00.000,2021-12-01 10:59:00.000,2020-06-01 10:00:00.000,NaN,2020-06-01 10:00:00.000,2020-06-01 10:00:00.000,2020-06-01 12:00:00.000,2021-11-30 12:00:00,HBO MAX,HBO MAX DOMESTIC,HBO MAX SUBSCRIPTION,0.0,2021-10-06 00:42:02.338,2021-10-06 00:42:02.338,2021-10
2,Dune,1,0,Popcorn,2021-10-22,2,7.549632e+18,urn:warnermedia:wmid:product:df59373b53ac93d41...,GYUjdLgBiJp5otAEAAAAJ,2239504,NaN,NaN,NaN,NaN,tt1160419,NaN,NaN,Dune,Dune,"Oscar nominee Denis Villeneuve (""Arrival,"" ""Bl...",A mythic and emotionally charged hero's journe...,teen,movie,adventure | drama | fantasy-sci-fi |

In [5]:
## Data availability for future titles 
def notnullsum(x):
    return x.notnull().sum()

df[df.category=='Popcorn'].groupby('premiere_year_month').agg({'title':'count', 
                                       'imdb_id': lambda x: notnullsum(x),
                                      'primary_genre_desc': lambda x: notnullsum(x),
                                      'release_year': lambda x: notnullsum(x),
                                      'license_contract_type': lambda x: notnullsum(x),
                                      'licensor': lambda x: notnullsum(x),
                                      'air_date': lambda x: notnullsum(x)})

,title,imdb_id,primary_genre_desc,release_year,license_contract_type,licensor,air_date
premiere_year_month,,,,,,,
2021-01,1,1,1,1.0,0,1,1
2021-02,2,2,2,2.0,0,2,2
2021-03,1,1,1,1.0,0,1,1
2021-04,2,2,2,2.0,0,2,2
2021-05,1,1,1,1.0,0,1,1
2021-06,3,2,2,2.0,0,2,2
2021-07,1,1,1,1.0,0,1,1
2021-08,2,2,2,2.0,0,2,2
2021-09,2,2,2,2.0,0,2,2


In [48]:
### Duplicate titles from different episode #, category (e.g. suicide squad in popcorn & pay1)
dup = df[df.duplicated(subset=['title','season','episode_number_in_series'])].sort_values(by='title')
print(dup.shape)
display(dup)

df.groupby(by=['title','episode_number_in_series']).sum()

(10, 100)


,title,tier,season,category,premiere_date,Unnamed: 0,asset_id,wm_id,viewable_id,source_asset_id,series_id,season_id,franchise_id,catalog_geo_id,imdb_id,csm_id,grace_notes_id,asset_title_long,asset_title_short,asset_summary_long,asset_summary_short,asset_tags_age,asset_tags_content_category,asset_tags_genre,asset_tags_other,brand_hubs,asset_type,catalog_asset_type,series_title_long,series_title_short,series_type,season_title_long,season_title_short,promoted_asset_id,season_number,season_premiere_ind,season_finale_ind,series_premiere_ind,episode_number_in_season,episode_number_in_series,program_type,content_category,primary_genre_desc,navigation_genre_desc,descriptive_genre_desc,wm_enterprise_genres,leaf_level_genres,animation_type,script_type,scripted_flag,sports_flag,kids_flag,international_flag,latino_flag,short_flag,age_group,original_network,release_year,license_contract_type,license_contract_term,licensor,original_language,first_offered_date_hbo,first_offered_date_now,first_offered_date_max,air_date,latest_availability_start_date,latest_availability_end_date,premiere_air_date,theatrical_release_date,original_linear_air_date,full_asset_tags_age,full_asset_tags_content_category,full_asset_tags_genre,full_asset_tags_other,full_brand_hubs,is_downloadable,asset_url,creation_time,last_modified_time,last_rollup_trigger_time,effective_start_date,effective_end_date,active_flag,title_id,viewable_id.1,start_utc,end_utc,latest_asset_release_date,season_first_offered_date,title_first_offered_date,first_offered_date,latest_availability_start_date.1,latest_availability_end_date.1,brand,territory,channel,ever_popcorn_title,creation_time.1,last_modified_time.1
619,Godzilla vs. Kong,1,0,Pay1,2021-08-15,619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,Lucas the Spider,3,3,Kids & Family,2022-06-16,248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,Mortal Kombat,1,0,Popcorn,2021-04-23,506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,Scavengers Reign,3,1,Scripted Drama Series,2022-10-13,363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796,Space Jam: A New Legacy,1,0,Pay1,2021-12-15,796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Stateless,3,0,Docu-Series,2022-05-24,411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

tier  season  Unnamed: 0  \
title                   episode_number_in_series                             
All That Glitters       5.0                          3       1         751   
Axios                   47.0                         3       4          86   
                        48.0                         3       4          85   
                        49.0                         3       4          87   
                        50.0                         3       4          90   
                        51.0                         3       4          84   
                        52.0                         3       4          88   
                        53.0                         3       4          89   
The Bridge              1.0                          3       1           6   
The Dog House: UK       9.0                          3       2         384   
                        10.0                         3       2         377   
                        11.0                         3       2         383   
                        12.0                         3       2         381   
                        13.0                         3       2         382   
                        14.0                         3       2         380   
                        15.0                         3       2         378   
                        16.0                         3       2         379   
The Shop: Uninterrupted 12.0                         3       4         723   
                        13.0                         3       4         725   
                        14.0                         3       4         724   
                        15.0                         3       4         726   
                        16.0                         3       4         727   

                                                      asset_id  franchise_id  \
title                   episode_number_in_series                               
All That Glitters       5.0                       2.480853e+18           0.0   
Axios                   47.0                      3.417718e+18           0.0   
                        48.0                      7.429259e+18           0.0   
                        49.0                      4.220720e+18           0.0   
                        50.0                      3.125906e+18           0.0   
                        51.0                      1.379632e+18           0.0   
                        52.0                      5.637143e+18           0.0   
                        53.0                      3.945143e+18           0.0   
The Bridge              1.0                       3.157351e+18           0.0   
The Dog House: UK       9.0                       2.291106e+17           0.0   
                        10.0                      4.470448e+18           0.0   
                        11.0                      8.574148e+18           0.0   
                        12.0                      4.731195e+17           0.0   
                        13.0                      6.691978e+18           0.0   
                        14.0                      5.906879e+18           0.0   
                        15.0                      6.957843e+18           0.0   
                        16.0                      6.867759e+18           0.0   
The Shop: Uninterrupted 12.0                      1.655524e+18           0.0   
                        13.0                      4.214459e+18           0.0   
                        14.0                      5.017164e+18           0.0   
                        15.0                      8.616970e+18           0.0   
                        16.0                      7.272415e+18           0.0   

                                                  catalog_geo_id  \
title                   episode_number_in_series                   
All That Glitters       5.0                                  0.0   
Axios                   47.0                        

## IMDB

In [166]:
querystr='''
select 
a.asset_title_long,
a.viewable_id,
a.imdb_id,
it.original_title,
it.title_id,
it.title_type,
it.genres,
imc.reference_type,
itr.original_title as reference_title,
itr.title_id as reference_title_id,
itr.title_type as reference_title_type,
imcr.reference_type as reference_referece_type
from max_prod.catalog.asset_dim a
join max_prod.catalog.reporting_asset_offering_dim raod
    on a.viewable_id = raod.viewable_id
left join enterprise_data.catalog.imdb_title it 
    on a.imdb_id = it.title_id
left join enterprise_data.catalog.imdb_movie_connection imc 
    on it.title_id = imc.title_id
left join enterprise_data.catalog.imdb_title itr 
    on itr.title_id = imc.reference_title_id
left join enterprise_data.catalog.imdb_movie_connection imcr
    on itr.title_id = imcr.title_id
    and imcr.reference_type in ('featured_in', 'spoofed_in')
where raod.brand = 'HBO MAX'
and raod.territory = 'HBO MAX DOMESTIC'
and raod.channel = 'HBO MAX SUBSCRIPTION'
and imc.reference_type in ('follows','spin_off_from','remake_of', 'version_of')
;
'''

cursor_list = ctx.execute_string(
    querystr
    )
df = pd.DataFrame.from_records(cursor_list[-1].fetchall(), columns=[x[0] for x in cursor_list[-1].description])
df.columns= df.columns.str.lower()
df.to_csv('s3://datascience-hbo-users/users/tjung/psi/imdb_features.csv')

# df = pd.read_csv('s3://datascience-hbo-users/users/tjung/psi/imdb_features.csv')
# display(df.head(2))
# display(df.shape)
# display(df.groupby(by='reference_type').count())

In [154]:
df

,asset_title_long,viewable_id,imdb_id,original_title,title_id,title_type,genres,reference_type,reference_title,reference_title_id,reference_title_type,reference_referece_type
0,Batman: Gotham Knight,GXdu2VgbP-KXCPQEAADfT,tt1117563,Batman: Gotham Knight,tt1117563,video,"[""Animation"",""Action"",""Crime"",""Sci-Fi"",""Thrill...",spin_off_from,Batman Begins,tt0372784,movie,featured_in
1,Batman: Gotham Knight,GXdu2VgbP-KXCPQEAADfT,tt1117563,Batman: Gotham Knight,tt1117563,video,"[""Animation"",""Action"",""Crime"",""Sci-Fi"",""Thrill...",spin_off_from,Batman Begins,tt0372784,movie,featured_in
2,Batman: Gotham Knight,GXdu2VgbP-KXCPQEAADfT,tt1117563,Batman: Gotham Knight,tt1117563,video,"[""Animation"",""Action"",""Crime"",""Sci-Fi"",""Thrill...",spin_off_from,Batman Begins,tt0372784,movie,featured_in
3,Batman: Gotham Knight,GXdu2VgbP-KXCPQEAADfT,tt1117563,Batman: Gotham Knight,tt1117563,video,"[""Animation"",""Action"",""Crime"",""Sci-Fi"",""Thrill...",spin_off_from,Batman Begins,tt0372784,movie,featured_in
4,Batman: Gotham Knight,GXdu2VgbP-KXCPQEAADfT,tt1117563,Batman: Gotham Knight,tt1117563,video,"[""Animation"",""Action"",""Crime"",""Sci-Fi"",""Thrill...",spin_off_from,Batman Begins,tt0372784,movie,featured_in
...,...,...,...,...,...,...,...,...,...,...,...,...
122220,The Champ,GXsVBnAg6m0CxkQEAAABB,tt0078950,The Champ,tt0078950,movie,"[""Drama"",""Sport""]",remake_of,The Champ,tt0021730,movie,featured_in
122221,The Champ,GXsVBnAg6m0CxkQEAAABB,tt0078950,The Champ,tt0078950,movie,"[""Drama"",""Sport""]",remake_of,The Champ,tt0021730,movie,featured_in
122222,The Champ,GXsVBnAg6m0CxkQEAAABB,tt0078950,The Champ,tt0078950,movie,"[""Drama"",""Sport""]",remake_of,The Champ,tt0021730,movie,spoofed_in
122223,The Champ,GXsVBnAg6m0CxkQEAAABB,tt0078950,The Champ,tt0078950,movie,"[""Drama"",""Sport""]",remake_of,The Champ,tt0021730,movie,spoofed_in


In [156]:
grpby=['asset_title_long','viewable_id','imdb_id','genres','title_type','reference_referece_type']
df_ref_ref = df.groupby(by=grpby).agg({'title_id':'count'}).reset_index()
df_ref_ref = df_ref_ref.pivot(index=grpby[:-1], columns='reference_referece_type', values='title_id')\
            .reset_index()


df_ref_ref

reference_referece_type,asset_title_long,viewable_id,imdb_id,genres,title_type,featured_in,spoofed_in
0,'Tis the Season to Be Smurfy,GX3Uk3gHbrBmDbAEAAAKX,tt0198267,"[""Animation"",""Short"",""Adventure"",""Comedy"",""Dra...",tvShort,40.0,30.0
1,12 Rounds 3: Lockdown,GXSycXQx6YZVLqQEAAA7P,tt3957956,"[""Action"",""Crime"",""Thriller""]",movie,6.0,NaN
2,2 Fast 2 Furious,GWbgztg3fzYRYRwEAAAE1,tt0322259,"[""Action"",""Crime"",""Thriller""]",movie,52.0,27.0
3,2010,GYS6mKgFBt6TCFwEAAABJ,tt0086837,"[""Adventure"",""Mystery"",""Sci-Fi"",""Thriller""]",movie,229.0,253.0
4,28 Weeks Later,GYM09_wI0UYahwwEAAABN,tt0463854,"[""Horror"",""Sci-Fi""]",movie,62.0,9.0
...,...,...,...,...,...,...,...
958,Zip Zip Hooray!,GX43N9ADxXGiAlAEAAAAK,tt0056720,"[""Animation"",""Family"",""Short"",""Comedy""]",short,103.0,2.0
959,Zipping Along,GXo0IGQ2ZDLDCYwEAABXB,tt0046567,"[""Animation"",""Family"",""Short"",""Comedy""]",short,28.0,2.0
960,Zoom and Bored,GXo0IGQxP7bLCwgEAABTh,tt0051228,"[""Animation"",""Family"",""Short"",""Comedy""]",short,67.0,2.0
961,Zoom at the Top,GX5wjuQ39niGvfwEAAAA6,tt0056723,"[""Animation"",""Family"",""Short"",""Comedy""]",short,98.0,2.0


In [172]:
### get imdb data for future titles.  
### features:  imdb_pg available; 
### identify if there're prequels:  reference_type = 'follows', 'remake_of','spin_off_from'
### identify popularity of prequels: 'featured_in'
df = df.fillna(0)

grpby=['asset_title_long','viewable_id','imdb_id','genres','title_type','reference_type']
df_ref = df.groupby(by=grpby).agg({'reference_title_id':'nunique'}).reset_index()
df_ref = df_ref.pivot(index=grpby[:-1], columns='reference_type', values='reference_title_id')\
            .reset_index()
df_ref = df_ref.rename(columns={'viewable_id':'title_id','follows':'ref_follows',
                                'spin_off_from':'ref_spin_off_from','remake_of':'ref_remake_of',
                               'version_of':'ref_version_of'})


grpby=['asset_title_long','viewable_id','imdb_id','genres','title_type','reference_referece_type']
df_ref_ref = df.groupby(by=grpby).agg({'title_id':'count'}).reset_index()
df_ref_ref = df_ref_ref.pivot(index=grpby[:-1], columns='reference_referece_type', values='title_id')\
            .reset_index()
df_ref_ref = df_ref_ref.rename(columns={'viewable_id':'title_id','featured_in':'ref_ref_featured_in','spoofed_in':'ref_ref_spoofed_in'})

grpby_title= ['tier','content_category','category','title_name', 'title_id','season_number',
        'program_type']
df_actuals=pd.read_csv('s3://datascience-hbo-users/users/tjung/psi/fv_actual_0922.csv')
df_actuals = df_actuals[['first_views'] + grpby_title].groupby(by=grpby_title).sum().reset_index()


col_ref = ['title_id', 'title_type', 'imdb_id', 'asset_title_long', 'genres','ref_follows','ref_spin_off_from','ref_remake_of', 'ref_version_of']
col_ref_ref = ['title_id','ref_ref_featured_in','ref_ref_spoofed_in']

df_tot = df_actuals.merge(df_ref[col_ref], how='left', on='title_id')\
                    .merge(df_ref_ref[col_ref_ref], how='left', on='title_id')
print(df_tot.shape)
display(df_tot.isnull().sum())
display(df_tot.describe())

display(df_tot.head(2))
df_tot = df_tot.fillna(0)


df_tot.loc[(df_tot['ref_follows']>5), 'ref_follows'] = 1
df_tot.to_csv('s3://datascience-hbo-users/users/tjung/psi/imdb_features_engineered.csv')



(329, 18)


tier                     0
content_category         0
category                 0
title_name               0
title_id                 0
season_number            0
program_type             0
first_views              0
title_type             297
imdb_id                297
asset_title_long       297
genres                 297
ref_follows            316
ref_spin_off_from      326
ref_remake_of          315
ref_version_of         319
ref_ref_featured_in    301
ref_ref_spoofed_in     305
dtype: int64

,tier,season_number,first_views,ref_follows,ref_spin_off_from,ref_remake_of,ref_version_of,ref_ref_featured_in,ref_ref_spoofed_in
count,329.000000,329.000000,3.290000e+02,13.000000,3.0,14.000000,10.00000,28.000000,24.000000
mean,2.571429,2.732523,6.195205e+04,5.000000,1.0,1.285714,10.20000,144.428571,39.250000
std,0.654654,8.900271,2.306117e+05,9.924717,0.0,0.468807,14.62722,304.866750,65.465059
min,1.000000,0.000000,0.000000e+00,1.000000,1.0,1.000000,1.00000,1.000000,1.000000
25%,2.000000,0.000000,1.271000e+03,1.000000,1.0,1.000000,1.25000,24.250000,6.000000
50%,3.000000,1.000000,5.578000e+03,1.000000,1.0,1.000000,4.00000,77.000000,13.000000
75%,3.000000,1.000000,2.707500e+04,3.000000,1.0,1.750000,9.75000,146.000000,28.000000
max,3.000000,51.000000,2.727097e+06,37.000000,1.0,2.000000,47.00000,1647.000000,234.000000


,tier,content_category,category,title_name,title_id,season_number,program_type,first_views,title_type,imdb_id,asset_title_long,genres,ref_follows,ref_spin_off_from,ref_remake_of,ref_version_of,ref_ref_featured_in,ref_ref_spoofed_in
0,1,movies,Pay1,Godzilla vs. Kong,GYFEzmwNES16GkQEAAAAC,0,acquired,45523,movie,tt5034838,Godzilla vs. Kong,"[""Action"",""Sci-Fi"",""Thriller""]",3.0,NaN,2.0,NaN,146.0,18.0
1,1,movies,Pay1,Mortal Kombat,GYGYKfQwaKLheqwEAAAEC,0,acquired,23747,movie,tt0293429,Mortal Kombat,"[""Action"",""Adventure"",""Fantasy"",""Sci-Fi"",""Thri...",NaN,1.0,1.0,NaN,176.0,62.0


In [ ]:
df[(df.asset_title_long.str.contains('The Matrix Reloaded')) & (df.reference_type=='featured_in')]

In [67]:
print(df_tot.columns)
display(df_tot[df_tot.category=='Popcorn'].groupby(by='reference_type').count())

display(df_tot['reference_title_id'].describe(percentiles=[0.25,0.5,0.75,0.8,0.9,0.95,0.99]))
display(df_tot[df_tot['category']=='Popcorn']['reference_title_id'].describe(percentiles=percents))
display(df_tot[(df_tot['category']=='Popcorn') & (df_tot['reference_type'].isin(list_ref))]['reference_title_id'].describe(percentiles=percents))


Index(['Unnamed: 0', 'title_id', 'title_name', 'season_number',
       'content_category', 'content_source', 'program_type', 'category',
       'tier', 'effective_start_date', 'request_date', 'premiere_ind',
       'asset_premiere_count', 'premiering_hours_runtime', 'first_views',
       'hours_viewed', 'days_since_premiere', 'days_on_platform',
       'finished_window_flag', 'asset_title_long', 'viewable_id', 'imdb_id',
       'genres', 'title_type', 'reference_type', 'reference_title_id'],
      dtype='object')


,Unnamed: 0,title_id,title_name,season_number,content_category,content_source,program_type,category,tier,effective_start_date,request_date,premiere_ind,asset_premiere_count,premiering_hours_runtime,first_views,hours_viewed,days_since_premiere,days_on_platform,finished_window_flag,asset_title_long,viewable_id,imdb_id,genres,title_type,reference_title_id
reference_type,,,,,,,,,,,,,,,,,,,,,,,,,
edited_from,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32
edited_into,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91
featured_in,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541
features,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225
followed_by,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32
follows,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161
referenced_in,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541,541
references,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477,477
remake_of,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64


count    25081.000000
mean         9.043978
std         19.055089
min          1.000000
25%          1.000000
50%          3.000000
75%          9.000000
80%         11.000000
90%         21.000000
95%         39.000000
99%        102.000000
max        180.000000
Name: reference_title_id, dtype: float64

count    2695.000000
mean       16.770315
std        34.740393
min         1.000000
1%          1.000000
5%          1.000000
10%         1.000000
25%         1.000000
50%         3.000000
75%        16.000000
90%        45.000000
95%        66.000000
99%       180.000000
100%      180.000000
max       180.000000
Name: reference_title_id, dtype: float64

count    734.000000
mean      11.096730
std       20.692649
min        1.000000
1%         1.000000
5%         1.000000
10%        1.000000
25%        1.000000
50%        3.000000
75%       12.000000
90%       21.000000
95%       26.000000
99%      102.000000
100%     102.000000
max      102.000000
Name: reference_title_id, dtype: float64

In [70]:
df_tot[(df_tot['category']=='Popcorn') & (df_tot.reference_type.isin(list_ref))]

,Unnamed: 0,title_id,title_name,season_number,content_category,content_source,program_type,category,tier,effective_start_date,request_date,premiere_ind,asset_premiere_count,premiering_hours_runtime,first_views,hours_viewed,days_since_premiere,days_on_platform,finished_window_flag,asset_title_long,viewable_id,imdb_id,genres,title_type,reference_type,reference_title_id
8645,5319,GYA79hQZbUsI3gQEAAAB0,The Little Things,0,movies,HBO Max,acquired,Popcorn,2,2021-01-29,2021-01-29,1,1,2.13,20408,950883.193,0,234,1,The Little Things,GYA79hQZbUsI3gQEAAAB0,tt10016180,"[""Crime"",""Drama"",""Mystery"",""Thriller""]",movie,references,3.0
8649,5320,GYA79hQZbUsI3gQEAAAB0,The Little Things,0,movies,HBO Max,acquired,Popcorn,2,2021-01-29,2021-01-30,0,0,0.00,109024,4390993.147,1,234,1,The Little Things,GYA79hQZbUsI3gQEAAAB0,tt10016180,"[""Crime"",""Drama"",""Mystery"",""Thriller""]",movie,references,3.0
8653,5321,GYA79hQZbUsI3gQEAAAB0,The Little Things,0,movies,HBO Max,acquired,Popcorn,2,2021-01-29,2021-01-31,0,0,0.00,106209,3745352.301,2,234,1,The Little Things,GYA79hQZbUsI3gQEAAAB0,tt10016180,"[""Crime"",""Drama"",""Mystery"",""Thriller""]",movie,references,3.0
8657,5322,GYA79hQZbUsI3gQEAAAB0,The Little Things,0,movies,HBO Max,acquired,Popcorn,2,2021-01-29,2021-02-01,0,0,0.00,52329,1747077.483,3,234,1,The Little Things,GYA79hQZbUsI3gQEAAAB0,tt10016180,"[""Crime"",""Drama"",""Mystery"",""Thriller""]",movie,references,3.0
8661,5323,GYA79hQZbUsI3gQEAAAB0,The Little Things,0,movies,HBO Max,acquired,Popcorn,2,2021-01-29,2021-02-02,0,0,0.00,26199,874480.246,4,234,1,The Little Things,GYA79hQZbUsI3gQEAAAB0,tt10016180,"[""Crime"",""Drama"",""Mystery"",""Thriller""]",movie,references,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41557,26208,GYGYKfQwaKLheqwEAAAEC,Mortal Kombat,0,movies,HBO Max,acquired,Popcorn,1,2021-04-23,2021-05-23,0,0,0.00,10429,383013.713,30,150,1,Mortal Kombat,GYGYKfQwaKLheqwEAAAEC,tt0293429,"[""Action"",""Adventure"",""Fantasy"",""Sci-Fi"",""Thri...",movie,remake_of,1.0
41558,26208,GYGYKfQwaKLheqwEAAAEC,Mortal Kombat,0,movies,HBO Max,acquired,Popcorn,1,2021-04-23,2021-05-23,0,0,0.00,10429,383013.713,30,150,1,Mortal Kombat,GYGYKfQwaKLheqwEAAAEC,tt0293429,"[""Action"",""Adventure"",""Fantasy"",""Sci-Fi"",""Thri...",movie,spin_off_from,1.0
41561,26209,GYGYKfQwaKLheqwEAAAEC,Mortal Kombat,0,movies,HBO Max,acquired,Popcorn,1,2021-04-23,2021-05-24,0,0,0.00,4060,136165.431,31,150,1,Mortal Kombat,GYGYKfQwaKLheqwEAAAEC,tt0293429,"[""Action"",""Adventure"",""Fantasy"",""Sci-Fi"",""Thri...",movie,references,12.0
41562,26209,GYGYKfQwaKLheqwEAAAEC,Mortal Kombat,0,movies,HBO Max,acquired,Popcorn,1,2021-04-23,2021-05-24,0,0,0.00,4060,136165.431,31,150,1,Mortal Kombat,GYGYKfQwaKLheqwEAAAEC,tt0293429,"[""Action"",""Adventure"",""Fantasy"",""Sci-Fi"",""Thri...",movie,remake_of,1.0
